In [6]:
import os
import cv2
import numpy as np

In [7]:
def load_dataset(dataset_path, img_size=(100, 100)):
    faces = []
    labels = []
    label_map = {}  # Mapping between label number and person name
    current_label = 0

    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_path):
            continue

        label_map[current_label] = person_name
        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue  # Skip any image that cannot be loaded
            img_resized = cv2.resize(img, img_size)  # Resize image
            faces.append(img_resized)
            labels.append(current_label)

        current_label += 1

    return faces, labels, label_map


In [8]:
# Load dataset
dataset_path = 'dataset/'
faces, labels, label_map = load_dataset(dataset_path)

# Convert lists to numpy arrays
faces = np.array(faces)
labels = np.array(labels)


In [9]:
# Initialize the recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Train the recognizer with the dataset
face_recognizer.train(faces, labels)


In [10]:
# Load the face detector (Haar Cascades)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces_detected = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Recognize faces
    for (x, y, w, h) in faces_detected:
        roi_gray = gray[y:y+h, x:x+w]
        label, confidence = face_recognizer.predict(roi_gray)

        # Display the name and confidence
        name = label_map[label]
        cv2.putText(frame, f'{name} ({confidence:.2f})', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()
